# Greenways and Trails

This is an indicator for the 2023 Virgin Islands and Puerto Rico.

Created by Amy Keister, last run by Amy Keister on Sept 15 2023. It took 40 minutes to run.

In [39]:
import os
import arcpy
import numpy

In [40]:
import time
start = time.time()

In [41]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\4_Indicators\CarGreenwaysAndTrails\GandT20230713.gdb"

In [42]:
# define final outputs
Out1 = r"D:\SE_Blueprint_2023\4_Indicators\CarGreenwaysAndTrails\CaribbeanGreenwaysAndTrails.tif"

In [43]:
# define sub-indicator outputs to help with user support

In [44]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [45]:
# define additional inputs
# highway IN ('footway', 'cycleway', 'bridleway', 'path')
trails= r"D:\SE_Blueprint_2023\4_Indicators\CarGreenwaysAndTrails\OpenStreetMap20230624\Greenways.gpkg\main.NotPrivateTrails"
sw= r"D:\SE_Blueprint_2023\4_Indicators\CarGreenwaysAndTrails\OpenStreetMap20230624\Greenways.gpkg\main.Sidewalks"

# The ccap data is served up seperately for PR and each USVI island
# I did some pre-processing in another notebook named CCAPpre_JustExportTif.ipynb because
# ArcPro is not working with the original download rasters for some reason. It won't ID them 
# correctly and it also 
# won't perform processes on them correctly. I've tried many things, most of them work in ArcPro 
# outside of phython. Outside of python, if I reproject and bring the output into a geodatabase, 
# then ArcPro starts recognizing these rasters. But when I tried this in python it didn't work. 
# I tried mosaicing them first but it didn't make a raster that ArcPro can deal with. Finally I was 
# able to get copy raster to create rasters that ArcPro will recognize, but even with that, I had 
# to reboot my machine to get this copy to work correctly in python.
ccapPR= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copyPR.tif"
ccapST= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copyST.tif"
ccapSJ= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copySJ.tif"
ccapSC= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copySC.tif"

### Start Analysis

In [46]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [47]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\4_Indicators\CarGreenwaysAndTrails\GandT20230713.gdb


### Pull out impervious data and reproject/rescale

Since the CCAP data is finer resolution than the NLCD, and is binary, I am testing converting it to 30 meters to make it more comaparable to the NLCD

In [10]:
# pull out impervious surface to make binary rasters
out_raster = arcpy.sa.Con(ccapPR, 100, 0, "Value = 2"); out_raster.save("ccapPRimp")
out_raster = arcpy.sa.Con(ccapST, 100, 0, "Value = 2"); out_raster.save("ccapSTimp")
out_raster = arcpy.sa.Con(ccapSJ, 100, 0, "Value = 2"); out_raster.save("ccapSJimp")
out_raster = arcpy.sa.Con(ccapSC, 100, 0, "Value = 2"); out_raster.save("ccapSCimp")

In [11]:
# Use focal statistics to calculate the percent of cells in a 30 meter square that are identified as impervious 
# in the CCAP data
with arcpy.EnvManager(outputCoordinateSystem=sr):
    out_raster = arcpy.sa.FocalStatistics("ccapPRimp", "Rectangle 30 30 MAP", "MEAN", "DATA", 90); out_raster.save("ccapPRimpFS")
    out_raster = arcpy.sa.FocalStatistics("ccapSTimp", "Rectangle 30 30 MAP", "MEAN", "DATA", 90); out_raster.save("ccapSTimpFS")
    out_raster = arcpy.sa.FocalStatistics("ccapSJimp", "Rectangle 30 30 MAP", "MEAN", "DATA", 90); out_raster.save("ccapSJimpFS")
    out_raster = arcpy.sa.FocalStatistics("ccapSCimp", "Rectangle 30 30 MAP", "MEAN", "DATA", 90); out_raster.save("ccapSCimpFS")

In [12]:
# reprojet and resample to 30 meter, the idea is that this will make something similar to the NLCD continuous 
# version of impervious surface
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.Resample("ccapPRimpFS", "ccapPRimpFSrs", "30 30", "NEAREST")
    arcpy.management.Resample("ccapSTimpFS", "ccapSTimpFSrs", "30 30", "NEAREST")
    arcpy.management.Resample("ccapSJimpFS", "ccapSJimpFSrs", "30 30", "NEAREST")
    arcpy.management.Resample("ccapSCimpFS", "ccapSCimpFSrs", "30 30", "NEAREST")

In [13]:
# use cell statistics to combine instead of mosaic because of overlap between SJ and ST
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics("ccapPRimpFSrs;ccapSTimpFSrs;ccapSJimpFSrs;ccapSCimpFSrs", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("IMP")

### Make 0/NoData distinction



In [14]:
# pull identify which areas were mapped with C-CAP
out_raster = arcpy.sa.SetNull(ccapPR, 1, "Value = 0"); out_raster.save("ccapPRmap")
out_raster = arcpy.sa.SetNull(ccapST, 1, "Value = 0"); out_raster.save("ccapSTmap")
out_raster = arcpy.sa.SetNull(ccapSJ, 1, "Value = 0"); out_raster.save("ccapSJmap")
out_raster = arcpy.sa.SetNull(ccapSC, 1, "Value = 0"); out_raster.save("ccapSCmap")

In [15]:
# reprojet and resample to 30 meter
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.Resample("ccapPRmap", "ccapPRmaprs", "30 30", "NEAREST")
    arcpy.management.Resample("ccapSTmap", "ccapSTmaprs", "30 30", "NEAREST")
    arcpy.management.Resample("ccapSJmap", "ccapSJmaprs", "30 30", "NEAREST")
    arcpy.management.Resample("ccapSCmap", "ccapSCmaprs", "30 30", "NEAREST")

In [16]:
# use cell statistics to combine instead of mosaic because of overlap between SJ and ST
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics("ccapPRmaprs;ccapSTmaprs;ccapSJmaprs;ccapSCmaprs", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("map")

### Explore Trail Data

Change trail projection to the projection we are using for this analysis

In [17]:
# convert all non-private trails to a geodatabase
with arcpy.EnvManager(extent=Rextent):
    arcpy.management.FeatureToLine(trails, "NotPrivateTrails", None, "ATTRIBUTES")

In [18]:
# reproject trails
with arcpy.EnvManager(outputCoordinateSystem=sr,extent=Rextent):
    arcpy.management.CopyFeatures("NotPrivateTrails", "Trails1", '', None, None, None)

### Identify trails that we believe are connected
There are gaps in the trails that are due to digitizing errors. If a trail is within 1 meter of another trail, we are assuming this is a digitizing error and not a real gap in the trail.

In [19]:
# buffer the trails by 1 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Buffer("Trails1", "TrailsBuff", "1 Meters", "FULL", "ROUND", "NONE", [], "PLANAR")

In [20]:
# dissolve the buffered trails. Should be able to do this as a part of the function above, but it tends to crash so
# I'm doing it seperately
arcpy.management.Dissolve("TrailsBuff", "TrailsBuffDis", [], [], "SINGLE_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarGreenwaysAndTrails\\GandT20230713.gdb\\TrailsBuffDis'>

In [21]:
# add and calculate a field to hold unique IDs for the buffered dissolved lines. This will be used to indentify trails
# that are connected
arcpy.management.CalculateField("TrailsBuffDis", "ID", "!OBJECTID!", "PYTHON3", "", "LONG")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarGreenwaysAndTrails\\GandT20230713.gdb\\TrailsBuffDis'>

In [22]:
# use a spatial join to attach the unique id from the buffered trails back to the original linework
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.SpatialJoin("Trails1", "TrailsBuffDis", "Trails2", "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "INTERSECT", "", "")

In [23]:
# dissolve trails to find length of connected trails
arcpy.management.Dissolve("Trails2", "Trails2Dis", "ID", "", "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarGreenwaysAndTrails\\GandT20230713.gdb\\Trails2Dis'>

In [24]:
# add and calculate length field
# this used to work, but it doesn't work anymore. 
#arcpy.management.CalculateField("Trails2Dis", "LengthM", "!shape.geom_Length@meters!", "PYTHON3", "", "DOUBLE")

In [25]:
# add and calculate length field
arcpy.management.CalculateGeometryAttributes("Trails2Dis", "LengthM LENGTH", "METERS", '', sr, "SAME_AS_INPUT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarGreenwaysAndTrails\\GandT20230713.gdb\\Trails2Dis'>

In [26]:
# create a raster based on the trail length value
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolylineToRaster("Trails2Dis", "LengthM", "TrailLength", "MAXIMUM_LENGTH", "NONE", Rextent)

In [27]:
# make an ISNull version of the trails length raster. This will be used later if we want to assign 0 to areas that are not trails
out_raster = arcpy.sa.IsNull("TrailLength"); out_raster.save("TrailLengthIsNull")

### Perform analysis to calcuate percent impervious near trails

Nearby impervious: is defined as the average impervious surface within a 300 m radius circle surrounding the path. HOWEVER along that 300 m stretch of trail, we only count the impervious surface within a 45 m buffer on either side of the trail, since pixels nearer the trail have a bigger impact on the greenway/trail experience. So really this is looking forward and behind a person 300 m as they walk on the trail.

Local impervious is defined as the percent impervious surface of the 30 m pixel that intersects the trail. 

In [28]:
# buffer all trails
arcpy.analysis.Buffer("Trails2Dis", "Trails2Dis_45mbuff", "45 Meters", "FULL", "ROUND", "NONE", "", "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarGreenwaysAndTrails\\GandT20230713.gdb\\Trails2Dis_45mbuff'>

In [29]:
# extract impervious within 45 meters around trails
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("IMP", "Trails2Dis_45mbuff"); out_raster.save("Imp45")

In [30]:
# use focal statistics to calculate what we are calling "nearby impervious"
out_raster = arcpy.sa.FocalStatistics("Imp45", "Circle 10 CELL", "MEAN", "DATA"); out_raster.save("FocalStat")

In [31]:
# reduce this down again to the pixels that intersect the trails, since those are the only pixels we are scoring
out_raster = arcpy.sa.ExtractByMask("FocalStat", "Trails2Dis"); out_raster.save("ImpNearby")

In [32]:
# extract impervious surfact pixels that intersect trails
out_raster = arcpy.sa.ExtractByMask("IMP", "Trails2Dis"); out_raster.save("ImpLocal")

In [33]:
# calculate mean of nearby and local impervous surface
out_raster= arcpy.sa.CellStatistics(["ImpNearby", "ImpLocal"], "MEAN", "DATA", "SINGLE_BAND"); out_raster.save("mean")

In [34]:
# use a nested conditional statement to classify the mean of the local and nearby impervious surface
out_raster= arcpy.sa.Con(arcpy.Raster("mean") >= 10, 1, arcpy.sa.Con(arcpy.Raster("mean") >1, 2, arcpy.sa.Con(arcpy.Raster("mean") <= 1, 3, 0))); out_raster.save("NatDef")

### Give a lower score trails identifed as sidewalks in the openstreetmap data
This uses the sidewalk tag from open street map

In [35]:
# convert all sidewalks to a geodatabase
with arcpy.EnvManager(extent=Rextent):
    arcpy.management.FeatureToLine(sw, "Sidewalks", None, "ATTRIBUTES")

In [36]:
# reproject sidewalks
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures("Sidewalks", "Sidewalks1", '', None, None, None)

In [37]:
# add and calculate a field to use to convert to raster
arcpy.management.CalculateField("Sidewalks1", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CarGreenwaysAndTrails\\GandT20230713.gdb\\Sidewalks1'>

In [38]:
# create a raster from sidewalks
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolylineToRaster("Sidewalks1", "raster", "SidewalkR", "MAXIMUM_LENGTH", "NONE", Rextent)

In [39]:
# make an ISNull version of the sidewalks
out_raster = arcpy.sa.IsNull("SidewalkR"); out_raster.save("SidewalkRIsNull")

### Make legend values

In [40]:
# use a nested conditional statement to assign legend values based on the mean impervious and trail length
out_raster= arcpy.sa.Con((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") >= 40000),7, \
             arcpy.sa.Con(((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") >= 5000) | (arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") >= 40000)),6, \
                          arcpy.sa.Con(((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") >= 1900) | (arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") >= 5000)|(arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") >= 40000)),5,\
                                       arcpy.sa.Con(((arcpy.Raster("mean") <= 1) & (arcpy.Raster("TrailLength") < 1900) | (arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") >= 1900)|(arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") >= 5000)),4,\
                                                    arcpy.sa.Con(((arcpy.Raster("mean") < 10) & (arcpy.Raster("TrailLength") < 1900) | (arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") >= 1900)),3,\
                                                                 arcpy.sa.Con(((arcpy.Raster("mean") >= 10) & (arcpy.Raster("TrailLength") < 1900)),2,1)))))) \
; out_raster.save("Part1")

In [41]:
# Step one of doing zero/NoData Reclassify to make NoData 1, this will let me to a time next
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify("Part1", "Value", "NODATA 1;2 2;3 3;4 4;5 5;6 6", "DATA"); out_raster.save("Part2")

In [42]:
# Step two of doing zero/NoData
out_raster= arcpy.sa.Times("Part2", "map"); out_raster.save("Part3")

In [43]:
# Set three of doing zero/NoData. Reclassify to make zeros
out_raster = arcpy.sa.Reclassify("Part3", "Value", "1 0;2 2;3 3;4 4;5 5;6 6", "DATA"); out_raster.save("Part4")

In [44]:
# make trails that are tagged as sidewalks a value of 1, regardless of impervious
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster= arcpy.sa.Con("SidewalkRIsNull", "Part4", "1", "Value = 1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent

In [52]:
# clip to extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("indicator1")

In [53]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("indicator1", Out1, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [4]:
Out= r"D:\SE_Blueprint_2023\5_IndicatorLegendColorFix\CaribbeanGreenwaysAndTrails.tif"

In [5]:
# set code block for next step

codeblock = """
def Reclass(v):
	if v == 7:
		return '7 = Mostly natural and connected for ≥40 km'
	elif v == 6:
		return '6 = Mostly natural and connected for 5 to <40 km or partly natural and connected for ≥40 km'
	elif v == 5:
		return '5 = Mostly natural and connected for 1.9 to <5 km, partly natural and connected for 5 to <40 km, or developed and connected for ≥40 km'
	elif v == 4:
		return '4 = Mostly natural and connected for <1.9 km, partly natural and connected for 1.9 to <5 km, or developed and connected for 5 to <40 km'
	elif v == 3:
		return '3 = Partly natural and connected for <1.9 km or developed and connected for 1.9 to <5 km'
	elif v == 2:
		return '2 = Developed and connected for <1.9 km'
	elif v == 1:
		return '1 = Sidewalk'
	elif v == 0:
		return '0 = Not identified as a trail, sidewalk, or other path'
"""

In [6]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_IndicatorLegendColorFix\\CaribbeanGreenwaysAndTrails.tif'>

In [50]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 7:
		return 0
	if Value == 6:
		return 105
	if Value == 5:
		return 162
	if Value == 4:
		return 201
	if Value == 3:
		return 225
	if Value == 2:
		return 234
	if Value == 1:
		return 226
	else:
		return 255
		
def Reclass2(Value):
	if Value == 7:
		return 24
	if Value == 6:
		return 2
	if Value == 5:
		return 17
	if Value == 4:
		return 65
	if Value == 3:
		return 116
	if Value == 2:
		return 169
	if Value == 1:
		return 226
	else:
		return 255
		
def Reclass3(Value):
	if Value == 7:
		return 137
	if Value == 6:
		return 141
	if Value == 5:
		return 137
	if Value == 4:
		return 118
	if Value == 3:
		return 75
	if Value == 2:
		return 22
	if Value == 1:
		return 38
	else:
		return 255
"""

In [51]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [ ]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)